In [123]:
import boto3
import pandas as pd
from currency_converter import CurrencyConverter
import numpy as np
from src.utils import google
from typing import Iterator
import os
import dotenv
dotenv.load_dotenv()

True

In [14]:
SMART_MONEY_TYPES = ['accelerator', 'entrepreneurship_program', 'incubator', 'startup_competition']

In [15]:
cb_organisations = pd.read_parquet('organizations.parquet')
funding_rounds = pd.read_parquet('funding_rounds.parquet')
investments = pd.read_parquet('investments.parquet')
investors = pd.read_parquet('investors.parquet')


## Wrangler code

In [ ]:
# class CrunchbaseWrangler:
#     """
#     This class helps linking CB companies to other data such as the
#     investment rounds (deals) and investors participating in the deals
#     """

#     def __init__(self, cb_data_path: str = None):
#         """
#         Sets up the CrunchbaseWrangler class

#         Args:
#             cb_data_path: Optional argument to specify the location of the
#                 Crunchbase data, if it is different from the default location;
#                 NB: Note that this changes a global variable.
#         """
#         # Tables from the database
#         self._cb_organisations = None
#         self._cb_funding_rounds = None
#         self._cb_investments = None
#         self._cb_investors = None
#         self._cb_category_groups = None
#         self._cb_organisation_categories = None
#         self._cb_people = None
#         self._cb_degrees = None
#         # Organisation categories (industries)
#         self._industries = None
#         self._industry_groups = None
#         self._industry_to_group = None
#         self._group_to_industries = None
#         # Set data path (optional)
#         if cb_data_path is not None:
#             cb.CB_PATH = cb_data_path

#     @property
#     def cb_organisations(self):
#         """Full table of companies (this might take a minute to load in)"""
#         if self._cb_organisations is None:
#             self._cb_organisations = cb.get_crunchbase_orgs()
#         return self._cb_organisations

#     @property
#     def cb_funding_rounds(self):
#         """Table with investment rounds (all the deals for all companies)"""
#         if self._cb_funding_rounds is None:
#             self._cb_funding_rounds = cb.get_crunchbase_funding_rounds()
#         return self._cb_funding_rounds

#     @property
#     def cb_investments(self):
#         """
#         Table with investments for each round (deal). Note that one deal can
#         have several investments pertaining to different investors
#         """
#         if self._cb_investments is None:
#             self._cb_investments = cb.get_crunchbase_investments()
#         return self._cb_investments

#     @property
#     def cb_investors(self):
#         """Table with investors"""
#         if self._cb_investors is None:
#             self._cb_investors = cb.get_crunchbase_investors()
#         return self._cb_investors

#     def get_funding_round_investors(self, funding_rounds: pd.DataFrame) -> pd.DataFrame:
#         """
#         Gets the investors involved in the specified funding rounds

#         Args:
#             Dataframe with organisations specified by 'org_id' and 'name', and
#             all their funding rounds (deals).

#         Returns:
#             Dataframe with extra columns specifying investment round details:
#                 'funding_round_name': Name of the funding round
#                 'investor_name': Name of the investor organisation
#                 'investor_id' Crunchbase organisation identifier
#                 'investor_type': Specifies if investor is a person or an organisation
#                 'is_lead_investor': Specifies whether the organisation is leading the round (value is 1 in that case)

#         """
#         return funding_rounds.merge(
#             (
#                 self.cb_investments[
#                     [
#                         "funding_round_id",
#                         "funding_round_name",
#                         "investor_name",
#                         "id",
#                         "investor_type",
#                         "is_lead_investor",
#                     ]
#                 ].rename(columns={"id": "investor_id"})
#             ),
#             on="funding_round_id",
#             how="left",
#         ).assign(
#             raised_amount=lambda x: x.raised_amount.fillna(0),
#             raised_amount_usd=lambda x: x.raised_amount_usd.fillna(0),
#         )

#     def get_organisation_investors(
#         self, cb_organisations: pd.DataFrame
#     ) -> pd.DataFrame:
#         """
#         Gets the investors and corresponding funding rounds for the specified organisations

#         Args:
#             cb_organisations: Data frame that must have a columns with crunchbase
#                 organisation ids and names

#         Returns:
#              Dataframe with extra columns specifying investors and the corresponding investment rounds
#              (see docs for get_funding_rounds and get_funding_round_investors for more information)

#         """
#         return pipe(
#             cb_organisations, self.get_funding_rounds, self.get_funding_round_investors
#         )

    


## Enrichment utils

In [129]:
def convert_currency(
    funding_df: pd.DataFrame,
    date_column: str,
    amount_column: str,
    currency_column: str,
    converted_column: str = None,
    target_currency: str = "GBP",
) -> pd.DataFrame:
    """
    Convert amount in any currency to a target currency using CurrencyConverter package.
    Deal dates should be provided in the datetime.date format
    NB: Rate conversion for dates before year 2000 is not reliable and hence
    is not carried out (the function returns nulls instead)

    Args:
        funding_df: A dataframe which must have a column for a date and amount to be converted
        date_column: Name of column with deal dates
        amount_column: Name of column with the amounts in the original currency
        currency_column: Name of column with the currency codes (eg, 'USD', 'EUR' etc)
        converted_column: Name for new column with the converted amounts

    Returns:
        Same dataframe with an extra column for the converted amount

    """
    # Column name
    converted_column = (
        f"{amount_column}_{target_currency}"
        if converted_column is None
        else converted_column
    )
    # Check if there is anything to convert
    rounds_with_funding = len(funding_df[-funding_df[amount_column].isnull()])
    df = funding_df.copy()
    if rounds_with_funding > 0:
        # Set up the currency converter
        Converter = CurrencyConverter(
            fallback_on_missing_rate=True,
            fallback_on_missing_rate_method="linear_interpolation",
            # If the date is out of bounds (eg, too recent)
            # then use the closest date available
            fallback_on_wrong_date=True,
        )
        # Convert currencies
        converted_amounts = []
        for _, row in df.iterrows():
            # Only convert deals after year 1999
            if (row[date_column].year >= 2000) and (
                row[currency_column] in Converter.currencies
            ):
                converted_amounts.append(
                    Converter.convert(
                        row[amount_column],
                        row[currency_column],
                        target_currency,
                        date=row[date_column],
                    )
                )
            else:
                converted_amounts.append(np.nan)
        df[converted_column] = converted_amounts
        # For deals that were originally in the target currency, use the database values
        deals_in_target_currency = df[currency_column] == target_currency
        df.loc[deals_in_target_currency, converted_column] = df.loc[
            deals_in_target_currency, amount_column
        ].copy()
    else:
        # If nothing to convert, copy the values and return
        df[converted_column] = df[amount_column].copy()
    return df

def get_org_funding_rounds(
    organisations: pd.DataFrame,
    funding_rounds: pd.DataFrame,
    org_id_column: str = 'id'
) -> pd.DataFrame:
    """
    Get the funding rounds for a specific organisation

    Args:
        org_id: Crunchbase organisation identifier
        funding_rounds: Dataframe with funding rounds
        investments: Dataframe with investments
        investors: Dataframe with investors

    Returns:
        Dataframe with the funding rounds for the specified organisation
    """
    # Get the funding rounds for the organisation
    return (
        organisations[[org_id_column, "name"]]
        .rename(columns={org_id_column: "org_id"})
        .merge(
            funding_rounds,
            how = 'left',
            on = 'org_id'
        )
    )

def get_funding_round_investors(funding_rounds: pd.DataFrame, investments, investors) -> pd.DataFrame:
    """
    Gets the investors involved in the specified funding rounds

    Args:
        Dataframe with organisations specified by 'org_id' and 'name', and
        all their funding rounds (deals).

    Returns:
        Dataframe with extra columns specifying investment round details:
            'funding_round_name': Name of the funding round
            'investor_name': Name of the investor organisation
            'investor_id' Crunchbase organisation identifier
            'investor_type': Specifies if investor is a person or an organisation
            'is_lead_investor': Specifies whether the organisation is leading the round (value is 1 in that case)

    """
    investments_cols = [
        "funding_round_id",
        "investor_id",
        "investor_name",
        "investor_type",
        "is_lead_investor",
        ]
    investor_cols = [
        "id",
        "investor_types",
        "cb_url",
    ]
    return (
        funding_rounds
        .merge(
            investments[investments_cols],
            on="funding_round_id",
            how="left",
        )
        .merge(
            investors[investor_cols].rename(columns={"id": "investor_id", "cb_url": "investor_url"}),
            on="investor_id",
            how="left",
        )
    )

In [138]:
def _enrich_smart_money(
    funding_rounds_enriched: pd.DataFrame,
    investments: pd.DataFrame,
    investors: pd.DataFrame,
) -> pd.DataFrame:
    """
    """
    # Fetch the manually curated smart money table
    smart_money_manual_df = google.access_google_sheet(
        os.environ['SHEET_ID_INVESTORS'],
        "investors"
    )
    return (
        funding_rounds_enriched
        .pipe(get_funding_round_investors, investments=investments, investors=investors)
        .assign(            
            # Check if investment was made by one of the smart money types
            smart_money_auto = lambda df: df.investor_types.str.contains('|'.join(SMART_MONEY_TYPES)),
            # Check if investment was made by one of the manually curated smart money investors
            smart_money_manual = lambda df: df.investor_url.isin(smart_money_manual_df['Crunchbase URL']),
            # Combine the two smart money columns
            smart_money = lambda df: df.smart_money_auto | df.smart_money_manual
        )
    )

def enrich_funding_rounds(
    funding_rounds: pd.DataFrame,
    investments: pd.DataFrame,
    investors: pd.DataFrame,
    funding_round_ids: Iterator[str] = None, 
    cutoff_year: int = 2000,
) -> pd.DataFrame:
    """
    Add funding round information to a table with CB organisations

    Args:
        cb_organisations: Data frame that must have a columns with crunchbase
            organisation ids and names

    Returns:
        Dataframe with organisations specified by 'org_id' and 'name', and
        all their funding rounds (deals). Some of the important columns include:
            'funding_round_id': unique round id
            'announced_on': date of the round
            'investment_type': the type of round (seed, series etc.)
            'post_money_valuation_usd': valuation of the company
            'raised_amount_usd': amount of money raised in the deal (in thousands)

    """
    # If no IDs are specified, assume all funding rounds are needed
    if funding_round_ids is None:
        funding_round_ids = funding_rounds.id.unique()

    return (
        funding_rounds.query("id in @funding_round_ids")
        # More informative column names
        .rename(columns={
            "id": "funding_round_id",
            "name": "funding_round_name",
        })
        # Remove really old funding rounds
        .query(f"announced_on > '{cutoff_year}'")
        .sort_values("announced_on")
        .assign(
            # Convert investment amounts to thousands
            raised_amount=lambda df: df["raised_amount"] / 1e+3,
            raised_amount_usd=lambda df: df["raised_amount_usd"] / 1e+3,
            # Convert date strings to datetimes
            announced_on_date=lambda df: pd.to_datetime(df["announced_on"]),
        )
        # Get years from dates
        .assign(year=lambda df: df["announced_on_date"].apply(lambda x: x.year))
        # Convert USD currency to GBP
        .pipe(
            convert_currency,
            date_column="announced_on_date",
            amount_column="raised_amount",
            currency_column="raised_amount_currency_code",
            converted_column="raised_amount_gbp",
            target_currency="GBP",
        )
        .pipe(
            _enrich_smart_money,
            investments=investments,
            investors=investors,
        )
    )


def _enrich_total_funding_gbp(
    organisations: pd.DataFrame,
    funding_rounds_enriched: pd.DataFrame,
    date_start: int = None,
    date_end:  str = None,
):
    """
    Get the total funding in GBP for a set of organisations

    Args:
        funding_rounds_enriched_df: Dataframe with enriched funding rounds
        date_start: Earliest date for the funding rounds to include, in the format 'YYYY-MM-DD'
        date_end: Latest date for the funding rounds to include, in the format 'YYYY-MM-DD'
    
    Returns:
        Dataframe with columns 'org_id' and 'total_funding_gbp'. Organisations that have no funding
        in the specified time period will not be included in the dataframe
    """
    # If no IDs are specified, assume all organisations are needed
    organisation_ids = funding_rounds_enriched.org_id.unique()

    # Determine the dates to include
    date_start = date_start or funding_rounds_enriched.announced_on.min()
    date_end = date_end or funding_rounds_enriched.announced_on.max()

    funding_rounds_df = (
        funding_rounds_enriched
        .loc[funding_rounds_enriched['announced_on'].between(date_start, date_end, inclusive='both')]
        .query("org_id in @organisation_ids")
        .groupby("org_id")
        .agg(
            total_funding_gbp=("raised_amount_gbp", "sum"),
            num_rounds=("funding_round_id", "count"),
        )
        .reindex(organisation_ids)
        .reset_index()
        .fillna(0)        
    )

    return (
        organisations
        .merge(
            funding_rounds_df,
            how = 'left',
            left_on='id',
            right_on='org_id'
        )
        .drop(columns='org_id')
    )   


In [139]:
def _process_max_employees(count_range: str) -> int:
    """
    Convert the employee count range to a single (highest) number

    Args:
        count_range: A string with the employee count range, eg, '51-100', '10000+'

    Returns:
        A single integer with the maximum number of employees
    """
    if count_range is None:
        return 0
    elif type(count_range) == str:
        return int(count_range.split('-')[-1].replace('+', ''))
    else:
        raise ValueError(f"Unexpected value for employee count: {count_range}")


def _enrich_investment_opportunity(
    organisations_enriched: pd.DataFrame
) -> pd.DataFrame:
    """
    """
    # Check for valid employee count and country
    valid_count_and_country = (
        (organisations_enriched.employee_count_max <= 100) 
        & (organisations_enriched.country_code == 'GBR')         
    )
    # Investment opportunities tags
    potential_investment_opp = (
        valid_count_and_country
        & (organisations_enriched.total_funding_gbp < 1e+3)
    )
    investment_opp = (
        valid_count_and_country
        & (organisations_enriched.total_funding_gbp >= 1e+3)
        & (organisations_enriched.total_funding_gbp <= 5e+3)
    )
    return (
        organisations_enriched
        .assign(
            potential_investment_opp = potential_investment_opp,
            investment_opp = investment_opp
        )
    )    


In [140]:
def enrich_organisations(
    organisations: pd.DataFrame,
    funding_rounds_enriched: pd.DataFrame,
    organisation_ids: Iterator[str] = None,
) -> pd.DataFrame:
    """
    """
     # If no IDs are specified, assume all organisations are needed
    organisation_ids = organisation_ids or organisations.id.unique()

    organisations_enriched = (
        organisations
        .query("id in @organisation_ids")
        .assign(employee_count_max = lambda df: df['employee_count'].apply(_process_max_employees))
        .pipe(
            _enrich_total_funding_gbp,
            funding_rounds_enriched=funding_rounds_enriched
        )
        .pipe(_enrich_investment_opportunity)
        .pipe(_enrich_org_smart_money, funding_rounds_enriched=funding_rounds_enriched)
    )
    return organisations_enriched


def _enrich_org_smart_money(
    organisations: pd.DataFrame,
    funding_rounds_enriched: pd.DataFrame,
) -> pd.DataFrame:
    """
    """
    # find orgs that have received smart money
    smart_money_orgs = (
        funding_rounds_enriched
        .query("smart_money")
        .org_id
        .unique()
    )
    return organisations.assign(smart_money = lambda df: df.id.isin(smart_money_orgs))


In [142]:
# Get funding rounds enriched
funding_rounds_enriched_df = enrich_funding_rounds(funding_rounds, investments, investors)

In [136]:
df = _enrich_smart_money(funding_rounds_enriched_df, investments, investors)
# rounds = get_org_funding_rounds(cb_organisations.head(100), funding_rounds_enriched_df)
# get_funding_round_investors(rounds, investments, investors)

,funding_round_id,funding_round_name,type,permalink,cb_url,rank,created_at,updated_at,country_code,state_code,...,raised_amount_gbp,investor_id,investor_name,investor_type,is_lead_investor,investor_types,investor_url,smart_money_auto,smart_money_manual,smart_money
155,f6b783d3-d94c-4fd1-8d85-0475c496a5d5,Non Equity Assistance - Precigenics,funding_round,precigenics-non-equity-assistance--f6b783d3,https://www.crunchbase.com/funding_round/preci...,43322.0,2023-12-18 10:33:10,2023-12-18 10:33:10,None,None,...,NaN,c2a2fb04-afc0-a82a-10c9-918c365e0a65,Eurasante,organization,None,"accelerator,incubator",https://www.crunchbase.com/organization/eurasante,True,False,True
269,4d3bf68b-8e43-448d-9d88-9b5ab0e7efa1,Pre Seed Round - PeopleLikeUs,funding_round,peoplelikeus-pre-seed--4d3bf68b,https://www.crunchbase.com/funding_round/peopl...,605994.0,2021-01-07 05:34:18,2021-02-12 14:55:26,CAN,QC,...,NaN,258eb70a-86ae-05ae-9bbf-5df6d956fb5e,ProMontreal Entrepreneurs,organization,None,incubator,https://www.crunchbase.com/organization/promon...,True,False,True
318,3e679fe5-c1b1-4dfd-916e-639d23378255,Venture Round - Net2u,funding_round,net2u-series-unknown--3e679fe5,https://www.crunchbase.com/funding_round/net2u...,315840.0,2020-04-29 17:38:22,2020-04-29 17:38:22,ESP,None,...,NaN,2000cdb5-7a16-6b86-0918-5f92ea7f5cc7,Going Investment S.A.,organization,True,"family_investment_office,incubator,private_equ...",https://www.crunchbase.com/organization/going-...,True,False,True
359,0b57a85f-bcaa-48a1-9da7-39b4819080f4,Venture Round - Autocity.com,funding_round,autocity-com-series-unknown--0b57a85f,https://www.crunchbase.com/funding_round/autoc...,216369.0,2020-04-29 18:07:52,2020-04-29 18:07:52,ESP,None,...,NaN,2000cdb5-7a16-6b86-0918-5f92ea7f5cc7,Going Investment S.A.,organization,True,"family_investment_office,incubator,private_equ...",https://www.crunchbase.com/organization/going-...,True,False,True
402,b72390b3-4653-1d78-3fd1-0e1cb003c5d6,Series D - Cogit.com,funding_round,cogit-com-series-d--b72390b3,https://www.crunchbase.com/funding_round/cogit...,389980.0,2016-01-04 11:27:42,2018-02-12 23:41:12,USA,CA,...,8645.162542,6c7d61e2-7d2b-e16b-f92d-27b521660235,Lighthouse Capital Partners,organization,False,incubator,https://www.crunchbase.com/organization/lighth...,True,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1164231,09092aac-a7b2-48bc-9736-e93e4a0dd23e,Series A - Podcastle,funding_round,podcastle-series-a--09092aac,https://www.crunchbase.com/funding_round/podca...,406.0,2024-02-02 14:36:38,2024-02-05 05:41:29,ARM,None,...,10633.604361,8541fff9-c49e-4979-8dcf-3dcae636d446,AI Fund,organization,None,"incubator,micro_vc,venture_capital",https://www.crunchbase.com/organization/andrew...,True,False,True
1164244,25ef83a1-8524-4ec3-b3b8-9d060425db21,Grant - Vivet Therapeutics,funding_round,vivet-therapeutics-grant--25ef83a1,https://www.crunchbase.com/funding_round/vivet...,1104.0,2024-02-01 08:57:35,2024-02-01 08:57:35,FRA,None,...,4177.250000,5adf83d7-237f-47cc-a2f7-a6b794e0bb6d,France 2030,organization,None,entrepreneurship_program,https://www.crunchbase.com/organization/france...,True,False,True
1164252,3e19b3c6-b14a-4acd-8d70-36863fc3fc47,Seed Round - Veo,funding_round,veo-1960-seed--3e19b3c6,https://www.crunchbase.com/funding_round/veo-1...,18139.0,2024-01-09 14:51:12,2024-01-09 14:51:12,GBR,None,...,1400.000000,2c30fc12-12da-22e1-0e3c-38cbd4ec7441,Innovate UK,organization,True,"entrepreneurship_program,venture_capital",https://www.crunchbase.com/organization/innova...,True,False,True
1164264,c83479d7-fc27-4508-8dfe-94d138deeeb7,Series B - NeoPhore,funding_round,neophore-series-b--c83479d7,https://www.crunchbase.com/funding_round/neoph...,285.0,2024-02-01 10:00:10,2024-02-01 10:03:11,GBR,None,...,9600.000000,a399869f-4ee8-5ffd-a82d-e5b898582972,Astellas Venture Management,organization,None,"corporate_venture_capital,entrepreneurship_pro...",https://www.crunchbase.com/organization/astell...,True

In [143]:
orgs_enriched = enrich_organisations(cb_organisations.head(10), funding_rounds_enriched_df)

In [144]:
orgs_enriched

,id,name,type,permalink,cb_url,rank,created_at,updated_at,legal_name,roles,...,alias2,alias3,primary_role,num_exits,employee_count_max,total_funding_gbp,num_rounds,potential_investment_opp,investment_opp,smart_money
0,e1393508-30ea-8a36-3f96-dd3226033abd,Wetpaint,organization,wetpaint,https://www.crunchbase.com/organization/wetpaint,160064.0,2007-05-25 13:51:27,2023-08-01 00:01:37,None,company,...,None,None,company,NaN,100,7.164357e+04,9.0,False,False,False
1,bf4d7b0e-b34d-2fd8-d292-6049c4f7efc7,Zoho,organization,zoho,https://www.crunchbase.com/organization/zoho,132132.0,2007-05-26 02:30:28,2023-10-19 07:46:32,Zoho Corporation Pvt. Ltd.,"investor,company",...,None,None,company,2.0,10000,NaN,NaN,False,False,False
2,5f2b40b8-d1b3-d323-d81a-b7a8e89553d0,Digg,organization,digg,https://www.crunchbase.com/organization/digg,25964.0,2007-05-26 03:03:23,2024-01-26 08:06:40,"Digg Holdings, LLC",company,...,None,None,company,NaN,100,9.165725e+04,19.0,False,False,True
3,f4d5ab44-058b-298b-ea81-380e6e9a8eec,Omidyar Network,organization,omidyar-network,https://www.crunchbase.com/organization/omidya...,42025.0,2007-05-26 03:21:34,2023-08-30 02:27:06,None,"investor,company",...,None,None,investor,62.0,250,NaN,NaN,False,False,False
4,df662812-7f97-0b43-9d3e-12f64f504fbb,Meta,organization,facebook,https://www.crunchbase.com/organization/facebook,55.0,2007-05-26 04:22:15,2023-11-27 12:05:27,"Meta Platforms, Inc.","investor,company",...,"Facebook, Inc.",FB,company,1.0,10000,1.751465e+07,30.0,False,False,False
5,7ca12f7a-2f8e-48b4-a8d1-1a33a0e275b9,Trinity Ventures,organization,trinity-ventures,https://www.crunchbase.com/organization/trinit...,54164.0,2007-05-26 20:13:00,2023-08-30 23:04:05,None,"investor,company",...,None,None,investor,144.0,50,NaN,NaN,False,False,False
6,b08efc27-da40-505a-6f9d-c9e14247bf36,Accel,organization,accel,https://www.crunchbase.com/organization/accel,6989.0,2007-05-26 20:27:39,2023-12-27 00:09:11,Accel Partners LP,"investor,company",...,None,None,investor,370.0,250,NaN,NaN,False,False,False
7,8d5c7e48-82da-3025-dd46-346a31bab86f,Meritech Capital Partners,organization,meritech-capital-partners,https://www.crunchbase.com/organization/merite...,53714.0,2007-05-26 20:29:59,2023-08-30 02:30:19,None,"investor,company",...,None,None,investor,130.0,50,NaN,NaN,False,False,False
8,60485007-8856-bbac-aa1b-c535c41f5f47,Omnidrive,organization,omnidrive,https://www.crunchbase.com/organization/omnidrive,733451.0,2007-05-27 10:25:32,2019-10-03 06:30:53,None,company,...,None,None,company,NaN,0,8.129266e+02,2.0,False,False,False
9,4111dc8b-c0df-2d24-ed33-30cd137b3098,Geni,organization,geni,https://www.crunchbase.com/organization/geni,205852.0,2007-06-01 02:52:34,2020-12-24 11:21:45,None,company,...,None,None,company,NaN,50,1.195691e+04,4.0,False,False,False


In [ ]:
# google.access_google_sheet(
#     "1-Sl2kglCa09r2Lg9CsGEKocc3m8pukOwwrbkQk7706I",
#     "ASF",
# )